In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

import utils.torchUtils as gnn

In [41]:
import torch
from torch import Tensor

from torch_geometric.data import Data
from torch_geometric.transforms import BaseTransform
from torch_geometric.utils import to_networkx
import networkx as nx

from torch_geometric.typing import Adj, PairTensor
from typing import Callable, Optional, Union, Tuple
from torch.nn import Linear, Module
from torch.nn.functional import softmax, relu, sigmoid, log_softmax, nll_loss

In [31]:
class ClusterY(BaseTransform):
    def __call__(self, data : Data) -> Data:
        data.cluster_y = (data.node_id + 3) // 4
        return data

In [32]:
from torch_geometric.loader import DataLoader

def load_dataset(fn='data/MX_1200_MY_500-training', template=None):
    dataset = gnn.Dataset(fn,transform=template.transform)
    training, testing = gnn.train_test_split(dataset[:3000], 0.33)
    training, validation = gnn.train_test_split(training, 0.5)

    trainloader = DataLoader(training, batch_size=50, shuffle=True, num_workers=8)
    validloader = DataLoader(validation, batch_size=50, shuffle=True, num_workers=8)
    testloader = DataLoader(testing, batch_size=50, shuffle=True, num_workers=8)

    return trainloader, validloader, testloader

template = gnn.Dataset('data/template',make_template=True, transform=ClusterY())
trainloader, validloader, testloader = load_dataset(template=template)

In [236]:
dataset = gnn.concat_dataset([f'data/{mass}-training' for mass in eightb.mass_list],transform=template.transform)

In [238]:
loader = DataLoader(dataset, batch_size=100, num_workers=8)

In [239]:
cluster_sums = torch.stack([torch.Tensor([ (data.cluster_y == i).sum() for i in range(3) ]) for data in loader]).sum(dim=0)
cluster_weights = cluster_sums.max()/cluster_sums

In [240]:
cluster_weights

tensor([1.2777, 1.0000, 1.0724])

In [227]:
from utils.torchUtils.LightningModel import LightningModel
from utils.torchUtils.cpp_geometric import layers

class GCNModel(torch.nn.Module):
    def __init__(self, n_in_node, n_in_edge, nn_conv1_out=64, nn_conv2_out=256, nn_linear_out=128, n_classes=5, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = layers.GCNConvMSG(n_in_node=n_in_node, n_in_edge=n_in_edge, n_out=nn_conv1_out)
        self.relu1 = layers.GCNRelu()
        self.conv2 = layers.GCNConvMSG(n_in_node=nn_conv1_out, n_in_edge=nn_conv1_out, n_out=nn_conv2_out)
        self.relu2 = layers.GCNRelu()

        self.linear1 = layers.GCNLinear(nn_conv2_out, nn_conv2_out, nn_linear_out)
        self.relu3 = layers.GCNRelu()

        self.linear_o = layers.GCNLinear(nn_linear_out, nn_linear_out, n_classes)

        self.log_softmax = layers.GCNLogSoftmax()

    def forward(self, data : Data) -> Tensor:
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x, edge_attr = self.conv1(x, edge_index, edge_attr)
        x, edge_attr = self.relu1(x, edge_index, edge_attr)

        x, edge_attr = self.conv2(x, edge_index, edge_attr)
        x, edge_attr = self.relu2(x, edge_index, edge_attr)

        x, edge_attr = self.linear1(x, edge_index, edge_attr)
        x, edge_attr = self.relu3(x, edge_index, edge_attr)

        node_o, edge_o = self.linear_o(x, edge_index, edge_attr)
        node_o, edge_o = self.log_softmax(node_o, edge_index, edge_o)
        return node_o, edge_o

class GoldenCluster(LightningModel):
    def __init__(self, nn_conv1_out=64, nn_conv2_out=256, nn_linear_out=128, n_classes=5, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = layers.GCNConvMSG(n_in_node=self.n_in_node, n_in_edge=self.n_in_edge, n_out=nn_conv1_out)
        self.relu1 = layers.GCNRelu()
        self.conv2 = layers.GCNConvMSG(n_in_node=nn_conv1_out, n_in_edge=nn_conv1_out, n_out=nn_conv2_out)
        self.relu2 = layers.GCNRelu()

        self.linear1 = layers.GCNLinear(nn_conv2_out, nn_conv2_out, nn_linear_out)
        self.relu3 = layers.GCNRelu()

        self.linear_o = layers.GCNLinear(nn_linear_out, nn_linear_out, 2)
        self.cluster_linear = torch.nn.Linear(nn_linear_out, 3)

    def forward(self, data : Data) -> Tensor:
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x, edge_attr = self.conv1(x, edge_index, edge_attr)
        x, edge_attr = self.relu1(x, edge_index, edge_attr)

        x, edge_attr = self.conv2(x, edge_index, edge_attr)
        x, edge_attr = self.relu2(x, edge_index, edge_attr)

        x, edge_attr = self.linear1(x, edge_index, edge_attr)
        x, edge_attr = self.relu3(x, edge_index, edge_attr)

        (node_o, edge_o), cluster_o = self.linear_o(x, edge_index, edge_attr), self.cluster_linear(x)
        node_o, edge_o, cluster_o = log_softmax(node_o, dim=-1), \
                                    log_softmax(edge_o, dim=-1), \
                                    log_softmax(cluster_o, dim=-1)
        return node_o, edge_o, cluster_o

    def shared_step(self, batch, batch_idx, tag=None):
        node_o, edge_o, cluster_o = self(batch)
        loss = gnn.node_losses.std_loss(self, node_o, batch) \
            +  gnn.edge_losses.std_loss(self, edge_o, batch) \
            +  self.type_weights[0]*nll_loss(cluster_o, batch.cluster_y, cluster_weights)

        return dict(loss=loss)
        

In [228]:
model = GoldenCluster(dataset=template)

In [229]:
node_o, edge_o, cluster_o = model(data)

In [231]:
from pytorch_lightning import Trainer

trainer = Trainer(gpus=0, max_epochs=10)
trainer.fit(model, trainloader, validloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type       | Params
----------------------------------------------
0 | conv1          | GCNConvMSG | 960   
1 | relu1          | GCNRelu    | 0     
2 | conv2          | GCNConvMSG | 49.4 K
3 | relu2          | GCNRelu    | 0     
4 | linear1        | GCNLinear  | 65.8 K
5 | relu3          | GCNRelu    | 0     
6 | linear_o       | GCNLinear  | 516   
7 | cluster_linear | Linear     | 387   
----------------------------------------------
117 K     Trainable params
0         Non-trainable params
117 K     Total params
0.468     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 42/42 [00:06<00:00,  6.30it/s, loss=10.6, v_num=100]


In [232]:
node_o, edge_o, cluster_o = model(data)

In [233]:
data.cluster_y, torch.exp(cluster_o).argmax(dim=-1)

(tensor([0, 2, 1, 2, 1, 1, 2, 1, 2, 0]),
 tensor([0, 1, 1, 2, 2, 0, 1, 2, 0, 0]))

In [223]:
cluster_o

tensor([[-0.3415, -1.3685, -3.3569],
        [-5.2739, -0.8987, -0.5314],
        [-4.4501, -0.5781, -0.8501],
        [-1.5686, -1.5274, -0.5542],
        [-3.4550, -0.6125, -0.8523],
        [-0.1841, -2.6553, -2.3239],
        [-2.8781, -0.6898, -0.8162],
        [-2.3770, -0.8621, -0.7238],
        [-1.7689, -1.0770, -0.7157],
        [-0.0876, -3.2841, -3.0714]], grad_fn=<LogSoftmaxBackward>)